[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Neuvork/Engeneering-thesis/blob/master/results.ipynb)

In [1]:
! git clone https://<username>:<password>@github.com/Neuvork/Engeneeringthesis.git --single-branch --branch cmaes

Cloning into 'Engeneeringthesis'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 80 (delta 46), reused 54 (delta 23), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [1]:
#DOPISAC CMA
#ZROBIC REKURENCYJNY ES
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm
import time
from IPython.display import clear_output
import copy
import cupy as cp
import tensorflow_datasets as tfds
import tensorflow as tf
from Engeneeringthesis.sigmas import Sigmas_Neural_Network
from Engeneeringthesis.NeuralNetwork import Neural_Network

In [2]:
!nvidia-smi

Tue Sep 22 18:30:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
def cuda_memory_clear():
    print("_total_bytes_before", mempool.total_bytes())
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()          
    print("_total_bytes_after", mempool.total_bytes())  

In [4]:
no_debug = 1
basic_debug_mode = 2
super_debug_mode = 3
only_interesting = 5
DEBUG_MODE = only_interesting

In [5]:
def generate_population():
    #300 - liczebnosc populacji
    raise exception("NEI WOLNO")
    mario_net = Neural_Network([('linear',(300, 28*28, 10))])
    mario_net_sigmas = sigmas_Mario_net([('linear',(300, 28*28, 10))])
    return (mario_net, mario_net_sigmas)

def best_population(population, population_scores, sigmas, population_size = 200000, children_size = 200000):
    if DEBUG_MODE % basic_debug_mode == 0:
      print("__best_population_start")
    population_size = population.layers[0][1].shape[0]
    children_size = population_size
    #population.move_to_cpu()
    new_marionet = Neural_Network(population.population_size,  population.input_size, population.input_layers)
    new_sigmas = Sigmas_Neural_Network(sigmas.sigmas_size,  sigmas.input_size, sigmas.input_layers)
    
    #new_marionet.move_to_cpu()
    for i in range(children_size):
        #participants = cp.random.choice( a = population_size, size = 2, replace = False)
        #chosen_one = cp.argmax(population_scores[participants])
        chosen_one = cp.argmax(population_scores)
        new_marionet.replace_individual(i, population.get_individual(chosen_one))
        new_sigmas.replace_individual(i, sigmas.get_individual(chosen_one))
    if DEBUG_MODE % basic_debug_mode == 0:
      print("__best_population_stop")
    return new_marionet, new_sigmas

def mutate_sigmas(sigmas, mutation_parameter_individual, mutation_parameter_coordinate):
  if DEBUG_MODE % basic_debug_mode == 0:
    print("SIGMAS MUTATION START", sigmas.layers_sigmas[0][1].shape)
  flag = 0
  random_individual_mutation = cp.random.normal(loc = 0., scale = mutation_parameter_individual, size = (sigmas.layers_sigmas[0][1].shape[0], 1, 1, 1, 1))
  for layer in sigmas.layers_sigmas:
      if layer[0]=='linear' and flag==0:
          flag=1
          random_individual_mutation = random_individual_mutation.reshape(sigmas.layers_sigmas[0][1].shape[0], 1 ,1)
      random_weight_mutation = cp.random.normal(loc = 0., scale = mutation_parameter_coordinate, size = layer[1].shape)
      layer[1] *= cp.exp(random_weight_mutation + random_individual_mutation).astype(cp.float32)
  if DEBUG_MODE % basic_debug_mode == 0:
    print("SIGMAS MUTATION STOP", sigmas.layers_sigmas[0][1].shape)

def mutate_network(population, sigmas):
  for j in range(len(population.layers)):
    population.layers[j][1] += cp.random.normal(0, sigmas.layers_sigmas[j][1], sigmas.layers_sigmas[j][1].shape)


def mutate_population(parents, sigmas, mutation_parameter_individual, mutation_parameter_coordinate):
    if DEBUG_MODE % basic_debug_mode == 0:
      print("__mutate_population_start")
    mutate_sigmas(sigmas, mutation_parameter_individual, mutation_parameter_coordinate)
    mutate_network(parents, sigmas)
    return parents, sigmas

def gen_new_population(population, population_sigmas, children, children_sigmas, population_scores, children_scores):
    if DEBUG_MODE % basic_debug_mode == 0:
      print("__gen_new_population_start")
    population_size = population.layers[0][1].shape[0]
    population_argsorted_scores = cp.argsort(-population_scores)
    children_argsorted_scores = cp.argsort(-children_scores)
    new_population = Neural_Network(population.population_size,  population.input_size, population.input_layers)
    new_sigmas = Sigmas_Neural_Network(population_sigmas.sigmas_size,  population_sigmas.input_size, population_sigmas.input_layers)
    

    new_scores = cp.zeros(population_size, dtype = cp.float32)

    population_pointer = 0 
    children_pointer = 0

    for i in range(population_size):
        if population_scores[population_argsorted_scores[population_pointer]] > children_scores[children_argsorted_scores[children_pointer]]:
            new_population.replace_individual(i, population.get_individual(population_argsorted_scores[population_pointer]))
            new_sigmas.replace_individual(i, population_sigmas.get_individual(population_argsorted_scores[population_pointer]))
            new_scores[i] = population_scores[population_argsorted_scores[population_pointer]]
            population_pointer+=1
        else:
            new_population.replace_individual(i, children.get_individual(children_argsorted_scores[children_pointer]))
            new_sigmas.replace_individual(i, children_sigmas.get_individual(children_argsorted_scores[children_pointer]))
            new_scores[i] = children_scores[children_argsorted_scores[children_pointer]]
            children_pointer+=1    

  
    #new_population.move_to_gpu()
    if DEBUG_MODE % basic_debug_mode == 0:
      print("__gen_new_population_stop")
    return new_population, new_sigmas, new_scores

def ES(population, sigmas, train_ds, iter_num=2, mutation_parameter_individual=.0001, mutation_parameter_coordinate=.0001):
    global best_indivudal_cupy
    population_size = population.layers[0][1].shape[0]
    children_size = population_size
    population_scores = evaluate_population(population, train_ds)
    best_results = []
    mean_results = []
    min_results = []

    sigmas_maxes = []
    sigmas_mins = []
    sigmas_means = []

    children_maxes = []
    children_mins = []
    children_means = []
    children_diff_from_best = []


    best_results.append(cp.max(population_scores))
    mean_results.append(cp.mean(population_scores))
    min_results.append(cp.min(population_scores))
    
    for i in range(iter_num):
        parents, parents_sigmas = best_population(population, population_scores, sigmas)
        children, children_sigmas = mutate_population(parents, sigmas, mutation_parameter_individual, mutation_parameter_coordinate)
        #children.move_to_gpu()
        cuda_memory_clear()
        children_scores = evaluate_population(children, train_ds)
        clear_output()
        print("BEST CHILDREN RESULT ", cp.max(children_scores))
        best_results.append(cp.max(children_scores))
        mean_results.append(cp.mean(children_scores))
        min_results.append(cp.min(children_scores))

        children_maxes.append(cp.max(population.layers[0][1][0]))
        children_mins.append(cp.min(population.layers[0][1][0]))
        children_means.append(cp.mean(population.layers[0][1][0]))
        children_diff_from_best.append(cp.mean(cp.abs(population.layers[0][1][0] - best_indivudal_cupy)))

        sigmas_maxes.append(cp.max(sigmas.layers_sigmas[0][1][0]))
        sigmas_mins.append(cp.min(sigmas.layers_sigmas[0][1][0]))
        sigmas_means.append(cp.mean(sigmas.layers_sigmas[0][1][0]))


        fig, axes = plt.subplots(2,2, figsize = (14,10))
        axes[0][0].plot(np.array(best_results), color = 'r')
        axes[0][0].plot(np.array(mean_results), color = 'g')
        axes[0][0].plot(np.array(min_results),  color  = 'b')
        axes[0][0].grid(True)
        axes[0][0].set_title('Results:')


        axes[0][1].plot(np.array(children_maxes), color = 'r')
        axes[0][1].plot(np.array(children_mins), color = 'g')
        axes[0][1].plot(np.array(children_means),  color  = 'b')
        axes[0][0].grid(True)
        axes[0][1].set_title('Weights:')

        #axes[1][0].plot(np.array(sigmas_maxes), color = 'r')
        axes[1][0].plot(np.array(sigmas_mins), color = 'g')
        axes[1][0].plot(np.array(sigmas_means),  color  = 'b')
        axes[0][0].grid(True)
        axes[1][0].set_title("Sigmas weights:")

        axes[1][1].plot(np.array(children_diff_from_best), color = 'r')
        axes[0][0].grid(True)
        axes[1][1].set_title('Distance from classically trained network')
        fig.tight_layout()
        plt.show()
        

        #population.move_to_gpu()
        #children.move_to_gpu()
        cuda_memory_clear()
        population, sigmas, population_scores = gen_new_population(population, sigmas, children, children_sigmas, population_scores, children_scores)
        cuda_memory_clear()
    return population, sigmas
    #return kozak_scores, mean_scores, worst_scores


def brute_dot(temp, lin):
    ret_temp = cp.zeros((temp.shape[0], lin.shape[2]))
    for i in range(temp.shape[0]):
        ret_temp[i] = cp.dot(temp[i], lin[i])
    return ret_temp


def evaluate_population(population, train_ds):
    create_input_time = 0
    preds_time = 0
    points_count_time = 0
    j  = 0
    if DEBUG_MODE % basic_debug_mode == 0:
      print("___EVALUATE_POPULATION_START")
    #scores = np.zeros(population.layers[0][1].shape[0], dtype = np.uint32)
    scores = cp.zeros(population.population_size, dtype = cp.uint32)
    for image, label in zip(cp.array(train_ds['image']), cp.array(train_ds['label'])):
        start = time.time()
        image = image.flatten()
        create_input_time += time.time() - start
        start = time.time()
        preds = population.forward(image)
        preds_time += time.time() - start
        start = time.time()
        #scores += cp.asnumpy(preds == label)
        scores += preds == label
        points_count_time += time.time() - start
        j += 1
      
    if DEBUG_MODE % basic_debug_mode == 0:
      print("___EVALUATE_POPULATION_STOP", "create_input_time: ", create_input_time, "preds_time:", preds_time,
          "points_count_time: ", points_count_time, "\n best result: ", np.max(cp.asnumpy( scores)),
          "mean socre: ", np.mean(cp.asnumpy( scores)), "min score: ", np.min(cp.asnumpy( scores))) 
    if DEBUG_MODE % only_interesting == 0:
      print("best result: ", np.max(cp.asnumpy( scores)), "mean socre: ", np.mean(cp.asnumpy( scores)), "min score: ", np.min(cp.asnumpy( scores)))

    return scores

In [6]:
POPULATION_SIZE = 5
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])
sigmas = Sigmas_Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])

__compute_dimensionality start
__compute_dimensionality stop  7840


In [7]:
train_ds_mnist = tfds.load("mnist", split = "train", shuffle_files=True, batch_size=-1)
test_ds_mnist = tfds.load("mnist", split = "test", shuffle_files=True, batch_size=-1)

train_ds_mnist = tfds.as_numpy(train_ds_mnist)
test_ds_mnist = tfds.as_numpy(test_ds_mnist)

train_ds_mnist = {"image" : cp.array(train_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(train_ds_mnist["label"]) }
test_ds_mnist = {"image" : cp.array(test_ds_mnist["image"]/255., dtype=cp.float32), "label" : cp.array(test_ds_mnist["label"]) }

In [8]:
!nvidia-smi

Tue Sep 22 18:30:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |    449MiB / 15079MiB |      8%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
class CMA_ES():
  def __init__(self,population,sigma,evaluate_func):
    self.dimensionality = population.dimensionality
    self.covariance_matrix = cp.diag(cp.ones(self.dimensionality, dtype = cp.float32))
    print("____allocated")
    cuda_memory_clear()
    self.population = population
    self.sigma = sigma
    self.isotropic = cp.zeros(self.dimensionality) #check it
    self.anisotropic = cp.zeros(self.dimensionality) #check it
    self.evaluate_func = evaluate_func
    self.dimensionality = population.dimensionality

  def _indicator_function(self, val, alpha):
    print("___indicator_function start ", val, alpha)
    if val < alpha * self.dimensionality and val > 0:
      print("___indicator_function stop ", 1)
      return 1
    else:
      print("___indicator_function stop ", 0)
      return 0
    return 0


  def update_mean(self, scores,sorted_indices,population,mu):
    print("___update_mean start")
    interesting_values = sorted_indices[:mu]
    valuable_individuals = cp.array(self.population.return_chosen_ones(interesting_values)) #maybve works
    normalized_interesting_values = interesting_values/cp.sum(interesting_values)
    print("valuable_individuals.shape ", valuable_individuals.shape)
    print("normalized_interesting_values.shape ", normalized_interesting_values)
    print("valuable_individuals: ", valuable_individuals)
    print("normalized_interesting_values: ", normalized_interesting_values)
    updated_mean = cp.mean(valuable_individuals * normalized_interesting_values.reshape(3, 1), axis = 0) #dont know if works
    print("___update_mean stop", updated_mean)
    return updated_mean
  
  def calculate_mu_w(self, scores,sorted_indices,mu):
    print("__calculate_mu_w start")
    interesting_values = sorted_indices[:mu]
    interesting_scores = scores[interesting_values]
    print("__calculate_mu_w stop")
    return 1. / cp.sum(interesting_scores**2)

  def update_isotropic(self,mean_act,mean_prev,c_sigma,mu_w):
    print("__update isotropic start, self.isotropic.shape = ", self.isotropic.shape)
    first_term = (1-c_sigma)*self.isotropic
    #q,r = cp.linalg.qr(self.covariance_matrix)
    #inversed_covariance_matrix = cp.linalg.inv(r).dot(q.T)
    inversed_covariance_matrix = cp.real(cp.array(sqrtm(cp.asnumpy(cp.linalg.inv(self.covariance_matrix)))))
    test = inversed_covariance_matrix.dot(inversed_covariance_matrix)
    inv_test = cp.linalg.inv(self.covariance_matrix)
    print("__difference without imaginary: ",cp.max(test-inv_test),cp.mean((test-inv_test).flatten()),cp.min(test-inv_test))
    second_term = cp.sqrt(1-((1-c_sigma)**2))*cp.sqrt(mu_w)
    print("mean_act ", mean_act)
    print("mean_prev ", mean_prev)
    print("self.sigma ", self.sigma)
    third_term = (cp.array(mean_act)-cp.array(mean_prev))/cp.array(self.sigma)
    print("__update isotropic stop, self.isotropic.shape = ", self.isotropic.shape)
    print("first term shape: ", first_term.shape)
    print("inversed_covariance_matrix.shape ", inversed_covariance_matrix.shape)
    print("second_term.shape ", second_term.shape)
    print("third_term.shape ", third_term.shape)
    print("inversed_covariance_matrix*second_term*third_term: ", (second_term*inversed_covariance_matrix.dot(third_term)))
    print("__first_term",first_term)
    print("second_term",second_term)
    print("third_term",third_term)
    print("mult_result",inversed_covariance_matrix.dot(third_term))
    print("inversed_covariance_matrix",inversed_covariance_matrix)
    return first_term + second_term*inversed_covariance_matrix.dot(third_term)
  
  def compute_cs(self, alpha, c_1, c_covariance):
    print("__compute_cs start")
    return (1 - self._indicator_function(cp.sqrt(np.sum(self.isotropic ** 2)), alpha)) * c_1 * c_covariance * (2 - c_covariance)
    print("__compute_cs stop")

  def update_anisotropic(self, mean_act,mean_prev,mu_w,c_covariance,alpha):
    print("__update_anisotropic start")
    print("____ self.isotropic.shape: ", self.isotropic.shape)
    ret_val = (1 - c_covariance) * self.anisotropic
    ret_val2 = self._indicator_function(self.norm(self.isotropic), alpha)
    ret_val2 *= np.sqrt(1 - (1 - c_covariance ** 2))
    ret_val2 *= np.sqrt(mu_w)
    ret_val3 = (mean_act - mean_prev) / self.sigma
    print("__update_anisotropic stop")
    return ret_val + ret_val2 * ret_val3
  
  def _sum_for_covariance_matrix_update(self, scores, sorted_indices, mu, mean_prev): #jakas almbda potrzebna chyba
    print("___sum_for_covariance_matrix_update start")
    interesting_values = sorted_indices[:mu]
    valuable_individuals = cp.array(self.population.return_chosen_ones(interesting_values))
    normalized_interesting_values = scores[interesting_values]/cp.sum(scores[interesting_values]) #wi sequence
    ret_sum = .0
    for i in range(mu):
      ret_sum += normalized_interesting_values[i] * np.dot((valuable_individuals[i] - mean_prev) #result should be matrix!!!
                / self.sigma, ((valuable_individuals[i] - mean_prev) / self.sigma).T  )
    print("___sum_for_covariance_matrix_update stop: ", ret_sum)
    return ret_sum


  def update_covariance_matrix(self, c_1, c_mu, c_s, scores, sorted_indices, mu, mean_prev):
    print("__update_covariance_matrix start")
    discount_factor = 1 - c_1 - c_mu + c_s
    C1 = discount_factor * self.covariance_matrix
    C2 = c_1 * (self.anisotropic.dot(self.anisotropic.T))
    C25 = self._sum_for_covariance_matrix_update(scores, sorted_indices, mu, mean_prev)
    C3 = c_mu * self._sum_for_covariance_matrix_update(scores, sorted_indices, mu, mean_prev)
    print("__shapeOfALL",C1.shape,C2.shape,C3.shape,C25.shape)
    print("__update_covariance_matrix stop")
    return C1 + C2 + C3

  def norm(self,vector):
    print("__norm vector.shape: ", vector.shape)
    print("__norm vector: ", vector)
    print("___ret val squared: ", vector.dot(vector.T))
    print("___ret val: ", cp.sqrt(vector.dot(vector.T)))
    
    return cp.sqrt(vector.dot(vector.T))

  def funkcja(self):
    return 0

  def update_sigma(self,c_sigma,d_sigma):
    print("_update_sigma start")
    temp = cp.sqrt(self.dimensionality)*(1-(1/(4*self.dimensionality)) + (1/(21*self.dimensionality**2)))
    temp2 = cp.exp((c_sigma/d_sigma)*((self.norm(self.isotropic)/temp)-1))
    print("_update_sigma stop")
    return self.sigma * temp2



     

  def fit(self, data, mu, lam, iterations): # mu is how many best samples from population, lam is how much we generate
    mean_act = cp.zeros(self.dimensionality)
    #constant
    c_sigma = 3/self.dimensionality
    d_sigma = 0.99 #dampening parameter could probably be hyperparameter, wiki says it is close to 1 so whatever
    c_covariance = 4/self.dimensionality
    alpha = 1.5
    c_1 = 2/(self.dimensionality**2)

    #body 
    for i in range(iterations):
      scores = self.evaluate_func(population, data)
      sorted_indices = cp.argsort(scores)
      mean_prev = mean_act.copy()
      self.population.parse_to_vector()
      mean_act = self.update_mean(scores,sorted_indices,population,mu) #we need to be vectorized here
      mu_w = self.calculate_mu_w(scores,sorted_indices,mu)
      print("____self.isotropic.shape: ", self.isotropic.shape)
      self.isotropic = self.update_isotropic(mean_act,mean_prev,c_sigma,mu_w)
      print("____self.isotropic.shape: ", self.isotropic.shape)
      c_s = self.compute_cs(alpha,c_1,c_covariance)
      print("____self.isotropic.shape: ", self.isotropic.shape)
      self.anisotropic = self.update_anisotropic(mean_act,mean_prev,mu_w,c_covariance,alpha)
      self.covariance_matrix = self.update_covariance_matrix(c_1,mu_w/(self.dimensionality**2),c_s,scores,sorted_indices,mu,mean_prev)
      self.sigma = self.update_sigma(c_sigma,d_sigma)
      self.population.sample(self.covariance_matrix, self.sigma, mean_act, lam)
      self.population.parse_from_vectors()
    return self.population

In [10]:
POPULATION_SIZE = 5
#input size do zmiany
population = Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])
sigmas = Sigmas_Neural_Network(POPULATION_SIZE,  (28*28, 1, 1), [['linear', 10, [1.,1.]]])

__compute_dimensionality start
__compute_dimensionality stop  7840


In [11]:
classifier = CMA_ES(population, 2., evaluate_population)

____allocated
_total_bytes_before 467118592
_total_bytes_after 466490880


In [ ]:
classifier.fit(train_ds_mnist, 3, 5, 5)

best result:  6405 mean socre:  5156.8 min score:  4493
_total_bytes_before 655131136
_total_bytes_after 466971136
__parse_to_vector before move to cuda
__parse_to_vector, self.vectorized
___update_mean start
valuable_individuals.shape  (3, 7840)
normalized_interesting_values.shape  [0.2 0.  0.8]
valuable_individuals:  [[ 2.2195294   1.2444795   1.5496544  ... -0.19251974  0.16045533
  -0.11612694]
 [ 0.5354378   1.9863014  -0.02132455 ... -1.6378549  -0.6834214
  -0.2977716 ]
 [-0.27190948  1.2296702  -0.12030191 ...  0.1701869  -0.77750987
  -1.0309142 ]]
normalized_interesting_values:  [0.2 0.  0.8]
___update_mean stop [ 0.07545943  0.41087735  0.07122978 ...  0.03254853 -0.19663894
 -0.28265225]
__calculate_mu_w start
__calculate_mu_w stop
____self.isotropic.shape:  (7840,)
__update isotropic start, self.isotropic.shape =  (7840,)
__difference without imaginary:  0.0 0.0 0.0
mean_act  [ 0.07545943  0.41087735  0.07122978 ...  0.03254853 -0.19663894
 -0.28265225]
mean_prev  [0. 0. 0

/usr/local/lib/python3.6/dist-packages/cupy/random/distributions.py:475: FutureWarning: cupy.random.multivariate_normal is experimental. The interface can change in the future.
  util.experimental('cupy.random.multivariate_normal')
/usr/local/lib/python3.6/dist-packages/cupy/random/generator.py:333: FutureWarning: cupy.random.RandomState.multivariate_normal is experimental. The interface can change in the future.
  util.experimental('cupy.random.RandomState.multivariate_normal')


_total_bytes_before 3180564992
_total_bytes_after 713335808
_total_bytes_before 3180564992
_total_bytes_after 713335808
_total_bytes_before 3180564992
_total_bytes_after 713335808
_total_bytes_before 3180564992
_total_bytes_after 713335808
_total_bytes_before 3180564992
_total_bytes_after 713335808
__sample stop
_total_bytes_before 713335808
_total_bytes_after 713021952
[5, 784, 10]
best result:  5923 mean socre:  5908.6 min score:  5851
_total_bytes_before 901662208
_total_bytes_after 713021952
__parse_to_vector before move to cuda
__parse_to_vector, self.vectorized
___update_mean start
valuable_individuals.shape  (3, 7840)
normalized_interesting_values.shape  [0.66666667 0.         0.33333333]
valuable_individuals:  [[ 2.8707216  -1.4257014   1.8377944  ...  0.8021144  -0.3939734
  -0.6803222 ]
 [-2.971014   -2.1462379  -0.1065572  ... -2.9297812  -0.04272245
   1.5969166 ]
 [-2.3088863  -3.2459836   0.0929036  ... -2.4512193   2.867754
  -1.3296688 ]]
normalized_interesting_values: 